# Workflow Test on Cosmos

### Testing the 'Sequence' Object in Python

In [6]:
#!/usr/bin/env python

import sys
sys.path.append('./modules') #to implement the modules in the 'modules' sub-directory
import Sequence
import tools
from Bio import SeqIO

sequences = []

for bioSeqRecord_obj in SeqIO.parse("data/test.fastq","fastq"):
    sequences.append(Sequence.Sequence(bioSeqRecord_obj))
print "BEFORE:"
print sequences[1].get_ID()
print sequences[1].get_SEQUENCE()
print sequences[1].get_LENGTH()


BEFORE:
SRR029695.24
AATCATCTTTCAAACCTATTGGTGCCTTTTCACCATATAATATATTGCCAACACACTAGCTAAAGCCATGAAA
73


In [7]:
print sequences[1].get_QUALITY()

[32, 26, 27, 25, 26, 27, 25, 32, 30, 12, 27, 33, 31, 17, 29, 22, 26, 24, 29, 22, 28, 21, 27, 27, 20, 10, 34, 32, 22, 10, 27, 27, 26, 18, 27, 27, 26, 27, 28, 21, 27, 27, 27, 27, 30, 23, 26, 24, 17, 29, 22, 27, 27, 26, 26, 27, 27, 27, 27, 27, 27, 33, 30, 12, 26, 27, 19, 27, 27, 27, 33, 30, 12]


**set_SEQUENCE()** function

In [8]:
sequences[1].set_SEQUENCE("AAAAAATTTTTAAAAGGGGCCCC")
print "AFTER:"
print sequences[1].get_SEQUENCE()
print sequences[1].get_LENGTH()

AFTER:
AAAAAATTTTTAAAAGGGGCCCC
23


**Quality treatment** test

In [9]:


tools.quality_treatment("data/test2.fastq","data/quality_treatment_results.fastq",15)

Results of the Quality treatment tool in data/quality_treatment_results.fastq

**Length treatment** test

In [10]:

tools.length_treatment("data/quality_treatment_results.fastq","testing/data/length_treatment_results.fastq",60)

**Formating files** test

In [11]:
tools.format_files("data/quality_treatment_results.fastq","data/format_test.txt",'quality')
tools.format_files("data/quality_treatment_results.fastq","data/format_test2.txt",'length')


**Rscripts** test

In [12]:
tools.r_call("plot_quality_scores.R","data/format_test.txt","results/test.pdf")
tools.r_call("plot_sequence_length.R","data/format_test2.txt","results/test2.pdf")
